## k-겹 교차 검증을 사용한 모델 성능 평가

모델에 대한 성능을 추정하는 단계는 머신러닝 모델을 구축하기 위한 핵심 단계이다.<br/>

홀드아웃 교차 검증(holdout cross-validation)은 가장 보편적인 성능 추정 방법이다.<br/>
홀드아웃 방법을 사용하는 가장 좋은 방법은 데이터를 훈련셋, 검증셋, 테스트셋으로 나누는 것이다.<br/>
훈련셋은 여러가지 모델을 훈련하는데 사용한다.<br/>
검증셋은 하이퍼파라메터를 튜닝할때 사용한다.(모델 선택 단계)<br/>
테스트셋은 모델의 성능을 추정하는데 사용한다.<br/>
홀드아웃의 단점은 검증셋의 성능 추정이 어떤 샘플을 사용하느냐에 따라서 민감하게 반응한다는 점이다.<br/>

<img src="../../images/holdout-cross-validation.png" width="700px" height="300px"/>
<caption><center>홀드아웃 교차 검증</center></caption>

k-겹 교차 검증 방법은 다음과 같다.<br/>
1. 데이터셋을 훈련셋과 테스트셋으로 나눈다.
2. 중복을 허락하지 않고 훈련셋을 k개의 폴드로 랜덤하게 나눈다.
3. k-1개의 폴드로 모델을 훈련하고, 나머지 하나의 폴드로 성능을 평가한다.
4. 이 과정을 k번 반복해서 k개의 모델과 성능 추정을 얻는다.
5. k개의 결과를 평균내서 최종 성능 추정(하이퍼파라미터)을 얻는다.

k-겹 교차 검증은 홀드아웃 방법에 비해서 훈련셋 분할의 영향을 덜받는다.<br/>
일반적으로 모델 튜닝에 k-겹 교차 검증이 사용된다.<br/>
k-겹 교차 검증으로 최적의 하이퍼파라미터를 찾고, 전체 훈련셋에 다시 모델을 훈련한다.<br/>
마지막으로 테스트셋을 사용해서 최종 성능 추정을 한다.<br/>

다음은 k=10일때, k-겹 교차 검증을 도식화한 것이다.<br/>

<img align="center" src="../../images/k-fold-cross-validation.png" width="700px" height="300px"/>
<caption><center>k-겹 교차 검증</center></caption>

훈련셋이 작을수록 k값을 크게, 훈련셋이 클수록 k값은 작게 설정한다.<br/>
k값이 너무 크면 실행 시간이 늘어나고 분산이 높은 추정을 만든다.<br/>
경험적으로 k=10이 가장 좋은 편향-분산 트레이드오프를 가진다.<br/>

k-겹 교차 검증은 클래스 비율이 동등하지 않을때 좋지않은 결과를 보인다.<br/> 
계층적 k-겹 교차 검증은 각 폴드의 클래스 비율이 훈련셋에 있는 클래스 비율을 대표하도록 유지한다.<br/> 
따라서 계층적 k-겹 교차 검증은 클래스 비율이 동등하지 않을때도 더 나은 편향과 분산 추정을 만든다.<br/>

다음은 계층적 k-겹 교차 검증으로 유방암 데이터셋을 사용한 로지스틱회귀 모델을 평가한다.

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

df = pd.read_csv(   # 위스콘신 유방암 데이터셋 로딩
    'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', 
    header=None
)

x = df.loc[:, 2:].values    # 30개의 특성을 x에 할당
y = df.loc[:, 1].values     # 클래스 레이블을 y에 할당
y = LabelEncoder().fit_transform(y) # 클래스 레이블을 정수로 변환(악성: 1, 양성: 0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, stratify=y, random_state=1)

pipe_lr = make_pipeline(    # 로지스틱회귀 모델 생성
    StandardScaler(),       # 표준화를 사용한 특성 스케일 조정
    PCA(n_components=2),    # PCA를 사용해서 2차원 부분공간으로 데이터 압축
    LogisticRegression(solver='liblinear', random_state=1)  # 로지스틱회귀 알고리즘으로 학습
)

In [6]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(
    n_splits=10,    # 폴드 개수
).split(x_train, y_train)
scores = []

for k, (train, test) in enumerate(kfold):   # train, test는 훈련셋과 테스트셋의 인덱스
    pipe_lr.fit(x_train[train], y_train[train])         # 모델을 훈련 
    score = pipe_lr.score(x_train[test], y_train[test]) # 모델의 정확도 점수를 계산
    scores.append(score)
    print('폴드: %2d, 클래스 분포: %s, 정확도: %.3f' % (k+1, np.bincount(y_train[train]), score))
    
print('\nCV 정확도: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

폴드:  1, 클래스 분포: [256 153], 정확도: 0.935
폴드:  2, 클래스 분포: [256 153], 정확도: 0.935
폴드:  3, 클래스 분포: [256 153], 정확도: 0.957
폴드:  4, 클래스 분포: [256 153], 정확도: 0.957
폴드:  5, 클래스 분포: [256 153], 정확도: 0.935
폴드:  6, 클래스 분포: [257 153], 정확도: 0.956
폴드:  7, 클래스 분포: [257 153], 정확도: 0.978
폴드:  8, 클래스 분포: [257 153], 정확도: 0.933
폴드:  9, 클래스 분포: [257 153], 정확도: 0.956
폴드: 10, 클래스 분포: [257 153], 정확도: 0.956

CV 정확도: 0.950 +/- 0.014


In [7]:
from sklearn.model_selection import cross_validate

# 훈련과 테스트에 걸린 시간과 지정한 평가 지표마다 훈련점수와 테스트 점수를 반환한다.
scores = cross_validate(    # k-겹 교차 검증 함수를 사용한 예
    estimator=pipe_lr, 
    X=x_train, 
    y=y_train, 
    scoring=['accuracy'],   # 평가 지표 선택
    cv=10,                  # 폴드 개수
    n_jobs=-1,              # CPU 코어 사용 개수(병렬처리)
    return_train_score=False    # 훈련 점수 반환 여부
)

# 반환된 딕션어리에서 훈련 점수와 테스트 점수를 추출하려면 'train_xxxx', 'test_xxxx'형식의 키를 사용한다.
accuracy = scores['test_accuracy']
print('CV 정확도 점수: %s' % accuracy)
print('CV 정확도: %.3f +/- %.3f' % (np.mean(accuracy), np.std(accuracy)))


CV 정확도 점수: [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]
CV 정확도: 0.950 +/- 0.014
